In [1]:
# -*- coding: utf-8 -*-
"""
   Introduction to Deep Learning (LDA-T3114)
   Skeleton Code for Assignment 1: Sentiment Classification on a Feed-Forward Neural Network

   Hande Celikkanat & Miikka Silfverberg
"""

'\n   Introduction to Deep Learning (LDA-T3114)\n   Skeleton Code for Assignment 1: Sentiment Classification on a Feed-Forward Neural Network\n\n   Hande Celikkanat & Miikka Silfverberg\n'

In [44]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import random

#ATTENTION: If necessary, add the paths to your data_semeval.py and paths.py here:
#import sys
#sys.path.append('</path/to/below/modules>')
from data_semeval import *
from paths import data_dir

In [79]:
#--- hyperparameters ---

N_CLASSES = len(LABEL_INDICES)
N_EPOCHS = 10
LEARNING_RATE = 0.05
BATCH_SIZE = 100
REPORT_EVERY = 1
IS_VERBOSE = True

In [80]:
def make_bow(tweet, indices):
    feature_ids = list(indices[tok] for tok in tweet['BODY'] if tok in indices)
    bow_vec = torch.zeros(len(indices))
    bow_vec[feature_ids] = 1
    return bow_vec.view(1, -1)

def generate_bow_representations(data):
    vocab = set(token for tweet in data['training'] for token in tweet['BODY'])
    vocab_size = len(vocab) 
    indices = {w:i for i, w in enumerate(vocab)}
  
    for split in ["training","development.input","development.gold",
                  "test.input","test.gold"]:
        for tweet in data[split]:
            tweet['BOW'] = make_bow(tweet,indices)

    return indices, vocab_size

# Convert string label to pytorch format.
def label_to_idx(label):
    return torch.LongTensor([LABEL_INDICES[label]])

In [94]:
#--- model ---

class FFNN(nn.Module):
    # Feel free to add whichever arguments you like here.
    def __init__(self, vocab_size, n_classes, extra_arg_1=None, extra_arg_2=None):
        super(FFNN, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(vocab_size, vocab_size),
            nn.ReLU(),
            nn.Linear(vocab_size, n_classes),
            nn.ReLU()
        )
        pass

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

In [82]:
#--- data loading ---
data = read_semeval_datasets(data_dir)
indices, vocab_size = generate_bow_representations(data)

In [95]:
#--- set up ---

# WRITE CODE HERE
model = FFNN(vocab_size, N_CLASSES) #add extra arguments here if you use
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)

In [ ]:
def to_tensor(s):
    if(s == 'neutral'):
        return torch.as_tensor([1, 0, 0])
    if(s == 'positive'):
        return torch.as_tensor([0, 1, 0])
    if(s == 'negative'):
        return torch.as_tensor([0, 1, 0])

#--- training ---
for epoch in range(N_EPOCHS):
    total_loss = 0
    # Generally speaking, it's a good idea to shuffle your
    # datasets once every epoch.
    random.shuffle(data['training'])    

    for i in range(int(len(data['training'])/BATCH_SIZE)):
        minibatch = data['training'][i*BATCH_SIZE:(i+1)*BATCH_SIZE]
        
        X = torch.stack([x['BOW'] for x in minibatch])
        y = torch.as_tensor([label_to_idx(y['SENTIMENT']) for y in minibatch])
        
        print(y)
        
        # Compute prediction error
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
           
        pass
    
    if ((epoch+1) % REPORT_EVERY) == 0:
        print('epoch: %d, loss: %.4f' % (epoch+1, total_loss*BATCH_SIZE/len(data['training'])))


tensor([0, 1, 1, 0, 2, 2, 2, 1, 1, 1, 2, 0, 2, 1, 2, 1, 2, 1, 1, 1, 2, 2, 1, 1,
        2, 0, 1, 2, 0, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 2, 1, 2,
        1, 1, 1, 1, 2, 1, 2, 0, 2, 1, 1, 1, 2, 0, 1, 1, 2, 0, 1, 2, 1, 2, 1, 2,
        1, 2, 2, 1, 2, 2, 2, 2, 2, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 1, 1, 1, 0, 1,
        2, 0, 1, 1])
tensor([1, 1, 2, 0, 1, 0, 2, 1, 2, 2, 1, 2, 2, 0, 1, 1, 2, 1, 0, 1, 1, 2, 2, 2,
        1, 1, 1, 0, 2, 1, 1, 2, 2, 2, 1, 1, 2, 2, 1, 1, 1, 1, 2, 1, 1, 0, 2, 0,
        1, 2, 0, 1, 1, 1, 2, 0, 1, 2, 2, 1, 0, 2, 2, 0, 1, 0, 1, 0, 2, 1, 2, 0,
        0, 2, 0, 1, 1, 0, 2, 2, 1, 1, 1, 1, 1, 1, 1, 1, 0, 2, 1, 1, 1, 1, 1, 2,
        0, 2, 0, 2])
tensor([1, 1, 1, 1, 1, 2, 0, 0, 2, 0, 1, 1, 2, 1, 2, 1, 2, 1, 0, 1, 2, 2, 2, 1,
        1, 0, 1, 2, 1, 1, 1, 2, 0, 1, 1, 1, 1, 1, 1, 1, 2, 1, 2, 1, 2, 1, 1, 2,
        2, 1, 1, 2, 2, 2, 2, 1, 1, 2, 1, 2, 1, 2, 2, 2, 1, 1, 2, 1, 2, 1, 1, 1,
        2, 1, 1, 0, 2, 2, 1, 1, 2, 1, 1, 1, 2, 0, 2, 1, 1, 1, 2, 1, 1, 2, 0, 1

tensor([1, 2, 1, 1, 1, 2, 1, 1, 1, 1, 0, 1, 1, 2, 1, 0, 1, 1, 2, 2, 1, 2, 0, 2,
        0, 1, 1, 1, 1, 1, 2, 0, 2, 1, 1, 1, 2, 1, 2, 2, 1, 1, 1, 1, 1, 2, 2, 1,
        1, 2, 1, 2, 1, 2, 1, 0, 2, 1, 1, 2, 2, 2, 1, 1, 0, 0, 1, 1, 0, 2, 1, 1,
        1, 2, 0, 1, 2, 2, 2, 1, 1, 2, 1, 1, 2, 1, 2, 2, 0, 1, 1, 1, 1, 2, 1, 1,
        2, 2, 2, 2])
tensor([0, 0, 2, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 2, 0, 1, 1, 0, 2,
        1, 1, 2, 0, 0, 2, 2, 1, 1, 1, 1, 1, 2, 2, 1, 1, 0, 1, 2, 1, 2, 2, 2, 1,
        1, 1, 1, 0, 1, 0, 1, 2, 1, 1, 0, 2, 2, 1, 2, 1, 1, 2, 1, 1, 2, 2, 0, 1,
        1, 2, 2, 2, 1, 1, 2, 1, 1, 1, 1, 1, 1, 0, 2, 1, 2, 1, 1, 2, 2, 0, 1, 1,
        1, 1, 1, 2])
tensor([1, 1, 2, 0, 2, 1, 1, 1, 2, 1, 1, 2, 1, 2, 2, 2, 2, 2, 2, 2, 0, 1, 1, 1,
        2, 2, 2, 0, 2, 1, 2, 1, 1, 1, 1, 2, 2, 1, 1, 2, 2, 1, 1, 1, 0, 1, 0, 2,
        2, 0, 2, 1, 2, 2, 0, 1, 1, 1, 2, 1, 2, 1, 1, 1, 2, 2, 0, 2, 2, 1, 2, 0,
        1, 2, 2, 1, 2, 0, 1, 1, 1, 2, 1, 2, 2, 1, 1, 1, 1, 0, 0, 0, 2, 1, 1, 1

In [ ]:
#--- test ---
correct = 0
with torch.no_grad():
    for tweet in data['test.gold']:
        gold_class = label_to_idx(tweet['SENTIMENT'])

        # WRITE CODE HERE
        # You can, but for the sake of this homework do not have to,
        # use batching for the test data.
        predicted = -1

        if IS_VERBOSE:
            print('TEST DATA: %s, GOLD LABEL: %s, GOLD CLASS %d, OUTPUT: %d' % 
                 (' '.join(tweet['BODY'][:-1]), tweet['SENTIMENT'], gold_class, predicted))

    print('test accuracy: %.2f' % (100.0 * correct / len(data['test.gold'])))